In [1]:
# %load mnist_loader.py

#### Libraries
# Standard library
import pickle 
##Esta libreria nos permite serializar objetos es decir convertir un objeto en base a 
#un lenguaje específico y deserializar que es el proceso análogo de string a objeto)
import gzip ##para comprimir y descomprimir archivo.zip como el mnist.pkl.gz

# Third-party libraries
import numpy as np## nos permite realizar los calculos con arrays

def load_data(): ##vamos a definir una función 

    f = gzip.open('mnist.pkl.gz', 'rb') #abre el archivo.zip, el modo rb, predeterminado como lectura de datos binarios) 
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")#definimos las variables a las que con
    #.load vamos a deserealizar y a dodificar en ascii global
    f.close()# cierra el mnist.pk
    return (training_data, validation_data, test_data)
#regresa las variables en datos de entrenamiento, datos de validación, y de prueba 

def load_data_wrapper():## pre procesamiento de los datos

    tr_d, va_d, te_d = load_data()##a estas variables les aplicamos la función
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]] ##reformamos una matriz de 784 x1 para la lista tr_d
    #comenzando por el elemento 0
    training_results = [vectorized_result(y) for y in tr_d[1]]##vamos a definir antes la función 
    training_data = zip(training_inputs, training_results)##como un ziper de una chamrra retorna un nuevo iterable 
    #cuyos elementos son tuplas que contienen un elemento de training data con otro de training results
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])#proceso análogo al zip para training data
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)#regresa los nuervos arrayas que formamos en la función.

def vectorized_result(j):##esta función toma un entero 
    e = np.zeros((10, 1))#matriz de ceros de 10x1
    e[j] = 1.0
    return e


In [5]:
# %load network.py



#### Libraries
# Standard library
import random # está libreria nos permite obtener datos aleatorios para alimentar 


# Third-party libraries
import numpy as np

class Network(object):##definimos la clase que será la neurona
    def __init__(self, sizes):##es la función con los parametros no de neuronas por capas y pesos

    
        self.num_layers = len(sizes)#tamaño de capas
        self.sizes = sizes
        self.biases =[np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)#esta funcion crea los pesos aleatoriamente
                        for x, y in zip(sizes[:-1], sizes[1:])]

def feedforward(self, a):
    for b, w in zip(self.biases, self.weights):
        a = sigmoid(np.dot(w, a)+b)##producto punto y le sumamos la bias
        return a

def SGD(self, training_data, epochs, mini_batch_size,momentum,eta,##obtiene los pesos para el backpropagation y evita que se atore
            test_data=None):
    training_data = list(training_data)##lista de datos para entrenar
    n = len(training_data)#nos da el tamaño de training data
    if test_data:
        test_data = list(test_data)
        n_test = len(test_data)
        for j in range(epochs):
            random.shuffle(training_data)##reorganizamos aleatoriamente los datos
            mini_batches = [
                training_data[k:k+mini_batch_size]#reparte los datos entre el númeroi de minibatches
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)##el.update le actualiza al minibatch
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test))#este ciclo nos permite visualizar 
                #datos o epocas
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):#la función que actualiza el minibatch
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]#llena de ceros
        #las listas que definimos se van llenando de la suma del grad por minibatch
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)#funcion de coste
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]#A los nuevos datos que obtuvimos del 
        #grad del minibatch  se les resta el valor del aprendizaje
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # lista de todas las activaciones de capa en capa
        zs = [] # definimos una lista a la que le vamos a meter los pesos por capa
        for b, w in zip(self.biases, self.weights):#ciclo para  ir juntando pesos y bias por capa
            z = np.dot(w, activation)+b
            zs.append(z)#añade
            activation = sigmoid(z)#aplica la función sigmoide a mi producto punto antes definido
            activations.append(activation)#añade a activations cada valor previamente calculado
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])#calcula el error de atrás hacía adelante es decir 
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):#errores para el gradiente 
            z = zs[-l]
            sp = sigmoid_prime(z)#derivada
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):

        test_results = [(np.argmax(self.feedforward(x)), y)#función para argumento mayor de la función feedforward para calcular 
                        #la salida de la red
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)#regersa la suma

    def cost_derivative(self, output_activations, y):##funcion de costo en relacion con las salidas
      
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))##función sigmoide

def sigmoid_prime(z):##derivada de la función sigmoide
    return sigmoid(z)*(1-sigmoid(z))

In [13]:
import mnist_loader ## importar código/bloque de los datos

training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 
training_data = list(training_data) 
import network #importar código de la red
 
net = network.Network([784, 30, 10]) ##parametros para entrenar
net.SGD(training_data, 55, 10, 0.07, test_data=test_data)

Epoch 0 : 3243 / 10000
Epoch 1 : 4137 / 10000
Epoch 2 : 4912 / 10000
Epoch 3 : 5185 / 10000
Epoch 4 : 5328 / 10000
Epoch 5 : 6092 / 10000
Epoch 6 : 6836 / 10000
Epoch 7 : 7029 / 10000
Epoch 8 : 7124 / 10000
Epoch 9 : 7235 / 10000
Epoch 10 : 7523 / 10000
Epoch 11 : 7718 / 10000
Epoch 12 : 7832 / 10000
Epoch 13 : 7880 / 10000
Epoch 14 : 7934 / 10000
Epoch 15 : 7970 / 10000
Epoch 16 : 8002 / 10000
Epoch 17 : 8027 / 10000
Epoch 18 : 8047 / 10000
Epoch 19 : 8072 / 10000
Epoch 20 : 8091 / 10000
Epoch 21 : 8101 / 10000
Epoch 22 : 8129 / 10000
Epoch 23 : 8134 / 10000
Epoch 24 : 8151 / 10000
Epoch 25 : 8175 / 10000
Epoch 26 : 8185 / 10000
Epoch 27 : 8194 / 10000
Epoch 28 : 8205 / 10000
Epoch 29 : 8223 / 10000
Epoch 30 : 8231 / 10000
Epoch 31 : 8235 / 10000
Epoch 32 : 8246 / 10000
Epoch 33 : 8253 / 10000
Epoch 34 : 8252 / 10000
Epoch 35 : 8270 / 10000
Epoch 36 : 8277 / 10000
Epoch 37 : 8276 / 10000
Epoch 38 : 8294 / 10000
Epoch 39 : 8302 / 10000
Epoch 40 : 8292 / 10000
Epoch 41 : 8306 / 10000
Ep

Creo que igual ya entendí que tal vez algo más conveniente era emplear el optimizador como una pieza del carro y no como el carro, pero en este caso:


In [32]:
import numpy as np
class AdamOptim(Network):
    def __init__(self, eta=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):#para inicializar
        self.m_dw, self.v_dw = 0, 0
        self.m_db, self.v_db = 0, 0
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.eta = eta
    def update(self, t, w, b, dw, db):#para actualizar pesos y sesgos
        self.m_dw = self.beta1*self.m_dw + (1-self.beta1)*dw
        self.m_db = self.beta1*self.m_db + (1-self.beta1)*db
        self.v_dw = self.beta2*self.v_dw + (1-self.beta2)*(dw**2)
        self.v_db = self.beta2*self.v_db + (1-self.beta2)*(db)

    
        m_dw_corr = self.m_dw/(1-self.beta1**t)
        m_db_corr = self.m_db/(1-self.beta1**t)
        v_dw_corr = self.v_dw/(1-self.beta2**t)
        v_db_corr = self.v_db/(1-self.beta2**t)

        w = w - self.eta*(m_dw_corr/(np.sqrt(v_dw_corr)+self.epsilon))
        b = b - self.eta*(m_db_corr/(np.sqrt(v_db_corr)+self.epsilon))
        return w, b
   

In [43]:
tam = [784, 30, 10]#tamaño para entrenar
optimizer = AdamOptim(tam) #para que podamos usar la clase y a los parametros de mi red
def loss_function(m):#definimos la función de perdida 
    return m ** 2 - 2 * m + 1

def grad_function(m):
    return 2 * m - 2

w = 0.5
b = -1.0
 

In [51]:
# Quedó raro pero ya no se me ocurre que más meterle, ahora solo conseguí que funcionara con sgd
# Iteraciones de entrenamiento
epochs = 55
for epoch in range(epochs):
    dw = grad_function(w_0)#para el gradiente
    db = grad_function(b_0)
    w_00 = w_0
    w_0, b_0 = adam.update(t,w=w_0, b=b_0, dw=dw, db=db)
    loss = loss_function(w) + loss_function(b)
    print(f'Epoch {epoch}: Loss {loss}')

Epoch 0: Loss 4.25
Epoch 1: Loss 4.25
Epoch 2: Loss 4.25
Epoch 3: Loss 4.25
Epoch 4: Loss 4.25
Epoch 5: Loss 4.25
Epoch 6: Loss 4.25
Epoch 7: Loss 4.25
Epoch 8: Loss 4.25
Epoch 9: Loss 4.25
Epoch 10: Loss 4.25
Epoch 11: Loss 4.25
Epoch 12: Loss 4.25
Epoch 13: Loss 4.25
Epoch 14: Loss 4.25
Epoch 15: Loss 4.25
Epoch 16: Loss 4.25
Epoch 17: Loss 4.25
Epoch 18: Loss 4.25
Epoch 19: Loss 4.25
Epoch 20: Loss 4.25
Epoch 21: Loss 4.25
Epoch 22: Loss 4.25
Epoch 23: Loss 4.25
Epoch 24: Loss 4.25
Epoch 25: Loss 4.25
Epoch 26: Loss 4.25
Epoch 27: Loss 4.25
Epoch 28: Loss 4.25
Epoch 29: Loss 4.25
Epoch 30: Loss 4.25
Epoch 31: Loss 4.25
Epoch 32: Loss 4.25
Epoch 33: Loss 4.25
Epoch 34: Loss 4.25
Epoch 35: Loss 4.25
Epoch 36: Loss 4.25
Epoch 37: Loss 4.25
Epoch 38: Loss 4.25
Epoch 39: Loss 4.25
Epoch 40: Loss 4.25
Epoch 41: Loss 4.25
Epoch 42: Loss 4.25
Epoch 43: Loss 4.25
Epoch 44: Loss 4.25
Epoch 45: Loss 4.25
Epoch 46: Loss 4.25
Epoch 47: Loss 4.25
Epoch 48: Loss 4.25
Epoch 49: Loss 4.25
Epoch 50: 

In [26]:
AdamOptim(Network)
